In [1]:
library(survival)
library(ggplot2)
library(dplyr)
library(reshape2)
library(stringr)

Warning message:
“package ‘survival’ was built under R version 4.0.5”
Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘reshape2’ was built under R version 4.0.5”
Warning message:
“package ‘stringr’ was built under R version 4.0.5”


In [2]:
dir_fig = 'R_clogit/for_publish_v9/'

file_dir = 'combined_tables_missing/'

#pca_combined = TRUE
#pca_combined = FALSE
file_pca_all = 'pca_calculation/PC_data_ALL.tsv'
file_pca_pb = 'pca_calculation/PC_data_PB.tsv'
file_pca_ont = 'pca_calculation/PC_data_ONT.tsv'
#if (pca_combined) {
#    pca_comb_suff = ''
#    pca_comb_title = 'pca: comb'
#} else {pca_comb_suff = '_pcaSepPBONT'
#       pca_comb_title = 'pca: sep PB/ONT'}

# for LongTR
#q_suf = '_Q'
# for snoopsv
#q_suf = ''

filter_samples = FALSE
#filter_samples = TRUE
#file_sam_missing = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/variant_prior/SV_process/sv_missingness/samples_missingness_30.tsv'
#file_sam_missing = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/variant_prior/SV_process/sv_missingness/samples_missingness_40.tsv'
file_sam_missing = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/variant_prior/SV_process/sv_missingness/samples_missingness_50.tsv'
# TR based missing
#file_sam_missing = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/TRs_LongTR/FAM/variant_prior/tr_missingness/samples_missingness_50.tsv'

#pca_flt_name = 'SQ20_SAMPLES'
pca_flt_name = 'SQ30_SAMPLES'
#pca_flt_name = 'SQ40_SAMPLES'
#pca_flt_name = 'SQ50_SAMPLES'
#pca_flt_name = 'SQ70_SAMPLES'

pca_mode = 'reg'
#pca_mode = 'null'

#use_gw_cov = FALSE
use_gw_cov = TRUE
use_gw_cov_ooc = FALSE
#use_gw_cov_ooc = TRUE
gw_suf = ''
gw_suf_ooc = ''
gw_var = ''
if (use_gw_cov) {
    gw_suf = '_useGWB'
    if (use_gw_cov_ooc) {
        gw_suf_ooc = '_OOC'
    }
    gw_var = 'count_genomewide'
}

In [10]:
### clean features
sub_fts = c('intergenic',
            'LOWPLI_intron', 'LOWPLI_cds + LOWPLI_utr', 
            'PLIp9_intron', 'PLIp9_cds + PLIp9_utr', 
            'DEV_BR_GENE_10_intron', 'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr')
sub_fts_col_names = c('intergenic', 
                  'lowpli_genes_intronic', 'lowpli_genes_exonic', 
                  'pli_genes_intronic', 'pli_genes_exonic', 
                  'fetal_brain_genes_intronic', 'fetal_brain_genes_exonic')

In [4]:
#prs_mode = 'sbayesr'
prs_mode = 'prscs'
file_suf = 'SQ70_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ70_SAMPLES_AD2_SAMPLES_HP12'
#file_suf = 'SQ70_SAMPLES_AD2_SAMPLES_NCOM'
#file_suf = 'SQ50_SAMPLES_AD2_SAMPLES'
file_suf_trmissing = '25'
#file_suf_trmissing = '20'
#file_suf_trmissing = '15'
if (prs_mode == 'sbayesr') {
    file_main = paste0(file_dir, 'combined_svs_trs_snvs_prs_pc_sbayesr_', file_suf, '_DELINS_GW', '_TRmissingness', file_suf_trmissing, '_fltHomPol_famGT.csv')
} else if (prs_mode == 'prscs') {
    file_main = paste0(file_dir, 'combined_svs_trs_snvs_prs_pc_prscs_', file_suf, '_DELINS_GW', '_TRmissingness', file_suf_trmissing, '_fltHomPol_famGT.csv')}

df_main <- read.table(file_main, header = TRUE, sep = ",")
#df_main['count_PLIp9_nonLOF_DEV_BR_10_cds'] = df_main[,'count_PLIp9_DEV_BR_GENE_10_cds'] - df_main[,'count_PLIp9_LOF_DEV_BR_10_cds']
df_main[,'Platform'] <- as.factor(df_main[,'Platform'])
df_main[,'sex'] <- as.factor(df_main[,'sex'])
dim(df_main)
df_main

[1]  243 5323

fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,⋯,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_del,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_ins,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_del,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_ins,PRS_ASD_Z,PRS_SCZ_Z,PRS_EDU_Z,count_dnlof_inhlof
<chr>,<chr>,<chr>,<fct>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
F0001,REACH000001,ASD,Male,1,trio,1,1,0.008627126,-0.005607061,⋯,0,0,0,0,0,0,-0.971701072,-0.510485305,0.82162924,0
F0026,REACH000026,ASD,Male,1,trio,1,0,0.095363540,-0.045229620,⋯,0,0,0,0,0,0,0.303644763,0.817056455,0.06312390,1
F0058,REACH000058,ASD,Male,1,trio,1,1,0.002421759,-0.009216429,⋯,0,0,0,0,0,0,0.595643472,-0.305436874,1.94303491,2
F0065,REACH000065,ASD,Male,1,trio,1,0,0.252773800,0.107932900,⋯,0,0,0,0,0,0,1.166191828,0.943709479,-2.78896425,1
F0078,REACH000086,ASD,Male,1,trio,2,0,0.044785250,0.038064880,⋯,0,0,0,0,0,0,0.703902177,-0.152554399,0.06482359,0
F0078,REACH000089,ASD,Female,1,trio,2,0,0.049879690,0.047204300,⋯,0,0,0,0,0,0,0.384605619,0.209070495,0.16440487,0
F0081,REACH000092,ASD,Female,1,trio,1,1,-0.028510680,-0.001001894,⋯,0,0,0,0,0,0,-0.898010204,-0.994547648,0.53088874,1
F0081,REACH000095,CON,Female,1,trio,1,1,-0.030440730,-0.002736118,⋯,0,0,0,0,0,0,-1.036073363,-1.487948561,0.29231934,1
F0083,REACH000097,ASD,Male,1,trio,1,0,0.095793110,0.052887850,⋯,0,0,0,0,0,0,0.817923575,-0.003587407,-0.08546761,1


In [5]:
# SV pca data
# using combined samples for pca
df_sv_pca_comb <- read.table(file_pca_all, header = TRUE, sep = "\t")
df_sv_pca_comb <- df_sv_pca_comb %>% select(SAMPLE, Platform, everything())

pcs = paste0('PC', 1:10, '_', pca_flt_name)
for (pc in pcs) {
    col_name = paste0(pc, '_comb')
    df_main[,col_name] <- df_sv_pca_comb[,pc]
}

# using separate PB/ONT samples for pca
df_pca_pb <- read.table(file_pca_pb, header = TRUE, sep = "\t")
df_pca_ont <- read.table(file_pca_ont, header = TRUE, sep = "\t")
#dim(df_pca_pb)
#dim(df_pca_ont)
# bring the SAMPLE and Platform column to the front
df_pca_pb <- df_pca_pb %>% select(SAMPLE, Platform, everything())
df_pca_ont <- df_pca_ont %>% select(SAMPLE, Platform, everything())
#dim(df_pca_pb)
#dim(df_pca_ont)
df_sv_pca <- rbind(df_pca_pb, df_pca_ont)
#dim(df_sv_pca)
# order the dataframe with the order of samples in df_main
df_sv_pca$SAMPLE <- factor(df_sv_pca$SAMPLE, levels=df_main$iid)
df_sv_pca <- arrange(df_sv_pca, SAMPLE)

#print('df_sv_pca:')
#df_sv_pca

pcs = paste0('PC', 1:10, '_', pca_flt_name)
for (pc in pcs) {
    df_main[,pc] <- df_sv_pca[,pc]
}
dim(df_main)

[1]  243 5343

In [6]:
### subset the main df to filter high missingness SVs

df_sam_missing = read.table(file_sam_missing, header = TRUE, sep = "\t", col.names = c('SAMPLE', 'PLAT'))
# for TR based filters
#df_sam_missing = read.table(file_sam_missing, header = TRUE, sep = "\t", col.names = c('SAMPLE'))
dim(df_sam_missing)

tmp = unlist(strsplit(file_sam_missing, '_'))
sample_missingness = unlist(strsplit(tmp[[length(tmp)]], '.tsv'))[[1]]
sample_missingness

flt_sam_suff = '_allSamples'
flt_sam_title = 'all samples'
if (filter_samples) {
    flt_sam_suff = paste0('_filterSamples', sample_missingness)
    flt_sam_title = paste('sample MS≤', as.double(sample_missingness)/100)
    # subset samples based on missingness
    df_main <- df_main[df_main$iid %in% df_sam_missing$SAMPLE,]
    # eliminate samples with low family members
    ##t = table(df_main$fid)
    ##df_main['fid_count'] = sapply(df_main$fid, function(x) t[x])
    ##df_main = df_main[df_main$fid_count>=3,]
}
dim(df_main)


df_main_ont = df_main %>% filter(Platform == 'ONT')
df_main_pb = df_main %>% filter(Platform == 'PB')
print(dim(df_main_ont))
print(dim(df_main_pb))

[1] 213   2

[1] "50"

[1]  243 5343

[1]  101 5343
[1]  142 5343


In [9]:
df_main['count_genomewide']

count_genomewide
<int>
336
309
231
276
184
186
244
248
336


In [ ]:
zs_thr = 3
supp_thr = 2
q_suf = '_Q'

gw_var_sv = 'count_genomewide'
gw_var_tr = paste0('count_ZS', zs_thr, '_genomewide_SUPP', supp_thr, '_Q')

cols_out = c('fid','iid','Phenotype','sex','MEAN_COVERAGE','Platform','rel','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',
             'count_dnlof_inhlof','PRS_ASD_Z', gw_var_sv, gw_var_tr)

df_out = df_main[cols_out]

for (i_ft in 1:length(sub_fts)) {
    ft = sub_fts[i_ft]
    col_out_base = sub_fts_col_names[i_ft]
    ft_tmp = unlist(strsplit(ft, split='+', fixed=TRUE)) # split vars with +
    ft_tmp = str_trim(ft_tmp) # trim spaces
    svs_tmp = paste0('count_', ft_tmp) # add count_
    trs_tmp = paste0('count_ZS', zs_thr, '_', ft_tmp, '_SUPP', supp_thr, q_suf) # add count_
    
    col_out = paste0('svs_', col_out_base)
    df_out[col_out] = 0
    for (ft_ in svs_tmp) {
        df_out[col_out] = df_out[col_out] + df_main[ft_]
    }

    col_out = paste0('trs_', col_out_base)
    df_out[col_out] = 0
    for (ft_ in trs_tmp) {
        df_out[col_out] = df_out[col_out] + df_main[ft_]
    }
}

df_out
file_out = paste0(dir_fig, 'burden_tests_table_counts.tsv')
print(file_out)
write.table(df_out, file = file_out, sep = "\t", row.names = FALSE, quote = FALSE)